## Análise da base de dados `Beta` utilizando algoritmos de ML

Neste notebook será analisado o `Beta dataset` utilizando algoritmos de ML para realizar a (1) extração de características, (2) seleção de características e (3) classificação dos dados

### Pontos importantes do dataset

- Frequências estimuladas (total de 40, com a diferença de 0.2 Hz uma da outra): 8.0, 8.2, ..., 15.6, 15.8;
- Taxa de amostragem: 250 Hz

### Analisar os "momentos" em que ocorrem evocação do sinal SSVEP

1. Criar o objeto `MNE` a partir dos dados dados do participante;
2. Aplicar no objeto `MNE` o filtro passa-faixa nos valores de 6 - 18 Hz;
3. Criar cópias do objeto `MNE` com fatias de tempo menores para analisar momentos que ocorrem estimulos ou não (verificar artigo);
    a) 0.0 - 0.5 segundos e 2.5 - 3.0 segundos ocorre apenas ruído;
    b) 0.5 - 2.5 segundos ocorre sinal SSVEP (com ruídos)
4. Com os sinais separados em objetos `MNE`, aplicar a `FFT`, para que seja possível plotar gráficos que contenham (ou não) as informações.
    - Os dados devem ser plotados no domínio da frequência (após a transformada de Fourier). O FFT pode ser realizado pela biblioteca `scipy.fft`.
    - Deve ser observado que as janelas (a) com ruído não aparecerão de fato o sinal SSVEP.

### Extração de características

Uma característica importante de acordo com o artigo base do dataset `BETA` é o *signal-to-noise ratio* (SNR).
São dois tipos de características SNR que podem ser implementadas: SNR de banda estreita (`narrow-SNR`) e SNR de banda larga (`wide-band SNR`).

Uma boa prática, é considerar o ruído das medidas de `SNR`, uma vez que os dados `SSVEP` não estão estimulados durante os períodos de 0 a 0,5 segundos e de 2,5 a 3 segundos. O ruído pode afetar a precisão das medidas de `SNR` e, portanto, é aconselhável levar isso em consideração.


In [203]:
# Lendo dados

import numpy as np
import mne

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.svm import SVC

dataMNE = mne.read_epochs("../../datasets/beta/beta-epo.fif", verbose= False)
data = np.load("../../datasets/beta/data_beta.npy")
labels = np.load("../../datasets/beta/labels_beta2.npy")

Agora iremos estimar o ruído de fundo, para calcular posteriormente o `narrow SNR` e o `wide-band SNR`. 

In [204]:
base_start = 0
base_end = 125
rest_start = 625
rest_end = 720

# armazena uma lista com as médias de potência para cada canal
noise_power = []
# consideramos a primeira amostra (1º target, 1º trial)
for channel_data in dataMNE.get_data()[0, :, :]:
    fft_result = np.fft.fft(channel_data)
    # densidade espectral de potência (PSD)
    psd = np.abs(fft_result) ** 2
    # média da potência nos intervalos de tempo sem estímulo
    base_power = np.mean(psd[base_start:base_end])
    rest_power = np.mean(psd[rest_start:rest_end])
    # média das duas médias de potência obtidas anteriormente
    mean_noise_power = (base_power + rest_power) / 2
    noise_power.append(mean_noise_power)
# média das médias de potência de todos os canais para estimar o ruído de fundo
estimated_background_noise = np.mean(noise_power)
estimated_background_noise

46372.03540036216

Antes de calcular os SNRs, precisamos obter as amplitudes alvo por meio dos dados EEG:

In [205]:
from scipy.signal import find_peaks

sr = 250

# frequências alvo
target_frequencies = np.arange(8, 16, 0.2)
# lista para armazenar as amplitudes nas frequências alvo
target_amplitudes = []

for channel_data in dataMNE.get_data():
    target = []
    for electrode in channel_data:
        fft_result = np.fft.fft(electrode)
        psd = np.abs(fft_result) ** 2
        frequencies = np.fft.fftfreq(len(fft_result), 1 / sr)
        target_amplitudes_trial = []
        for target_frequency in target_frequencies:
            # encontrando o índice da frequência alvo no espectro de frequência
            index = np.argmin(np.abs(frequencies - target_frequency))
            # amplitude na frequência alvo
            amplitude = np.sqrt(psd[index])
            target_amplitudes_trial.append(amplitude)

        target.append(target_amplitudes_trial)
        
    target_amplitudes.append(target)

target_amplitudes = np.array(target_amplitudes)
target_amplitudes.shape

(160, 64, 40)

Vamos calcular o SNR de "banda estreita". Pode ser observado pela seguinte equação:

$SNR_{banda\ estreita} = 10 \cdot \log_{10}\left(\frac{\text{energia total do espectro}}{\text{média das amplitudes nas frequências vizinhas}}\right)$

Já o SNR de banda larga é definido da seguinte forma:

$SNR_{banda\ larga} = 10 \cdot \log_{10}\left(\frac{\text{energia total do espectro}}{\text{energia total do espectro de amplitude}}\right)$

In [206]:
# forçando (estragando) valor de "estimated_background_noise" para não sobrar valores negativos
# estimated_background_noise = 1.
target_amplitudes_adjusted = target_amplitudes - estimated_background_noise

# subtraindo o ruído de fundo das amplitudes
narrow_band_SNR = 10 * np.log10(target_amplitudes_adjusted / estimated_background_noise)
# print(narrow_band_SNR)
print(narrow_band_SNR.shape)

total_power = np.sum(target_amplitudes_adjusted)
wide_band_SNR = 10 * np.log10(target_amplitudes_adjusted / total_power)
# print(wide_band_SNR)
print(wide_band_SNR.shape)

X = np.array([narrow_band_SNR, wide_band_SNR])
X = np.swapaxes(X, 0, 1)
X = np.swapaxes(X, 1, 2)
X = X.reshape(X.shape[0], X.shape[1], X.shape[2] * X.shape[3])
print(X.shape)


(160, 64, 40)
(160, 64, 40)
(160, 64, 80)


Ao final desta etapa, será obtido um vetor de características. Estas podem ser:
- `narrow SNR` (brigatória);
- `wide-band SNR` (brigatória);
- Maior valor espectral (FFT);
- Média dos valores espectrais (FFT).

Dimensionalidade dos dados será explicada da seguinte forma:

`40, 4, 64, 750` -> 40 targets, 4 trials, 64 canais e 750 valores
`160, 64 (SNR) + 64 (média) + 64 (maior) ...`
Resultando em `160, 192`.

### Seleção de características e classificação

Como existem diversos eletrodos (canais) que não obtém sinal SSVEP, podemos extrair as caracteríscas que não contribuem para a classificação dos dados.

Podemos utilizar o método `RFE` (*Recursive Feature Elimination*) aplicado por meio de `sklearn.feature_selection.RFE`, aprimorando o parâmetro `n_features_to_select` até obter o melhor resultado de classificação.

Para a classificação propriamente dita, é considerado o uso do método `SVM`.

In [207]:
# TAREFA 3
import matplotlib.pyplot as plt

channels = list(np.load("../../datasets/beta/channels_beta.npy"))
print(channels)

# data = mne.read_epochs("../../datasets/beta/beta-epo.fif")
channelList = ["PZ", "PO3", "PO5", "PO4", "PO6", "POZ", "O1", "OZ", "O2"]
index_remove = np.where(np.isin(channels, channelList, invert=True))
rest = [item for item in channels if item not in channelList]

X_new = np.delete(X, index_remove, axis=1)
print(X_new.shape, X.shape)

X_reshape = X_new.reshape(X_new.shape[0], X_new.shape[1] * X_new.shape[2])
print(X_reshape.shape)

['FP1', 'FPZ', 'FP2', 'AF3', 'AF4', 'F7', 'F5', 'F3', 'F1', 'FZ', 'F2', 'F4', 'F6', 'F8', 'FT7', 'FC5', 'FC3', 'FC1', 'FCZ', 'FC2', 'FC4', 'FC6', 'FT8', 'T7', 'C5', 'C3', 'C1', 'CZ', 'C2', 'C4', 'C6', 'T8', 'M1', 'TP7', 'CP5', 'CP3', 'CP1', 'CPZ', 'CP2', 'CP4', 'CP6', 'TP8', 'M2', 'P7', 'P5', 'P3', 'P1', 'PZ', 'P2', 'P4', 'P6', 'P8', 'PO7', 'PO5', 'PO3', 'POZ', 'PO4', 'PO6', 'PO8', 'CB1', 'O1', 'OZ', 'O2', 'CB2']
(160, 9, 80) (160, 64, 80)
(160, 720)


In [208]:

X_SS = StandardScaler().fit_transform(X_reshape)
y = LabelEncoder().fit_transform(labels)

imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
X_SS = imputer.fit_transform(X_SS)

X_train, X_test, y_train, y_test = train_test_split(X_SS, y, test_size=0.3, random_state=42)

clf = SVC(kernel='linear', C=1, probability=True, random_state=42).fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(accuracy_score(y_test, y_pred))

0.0625


In [229]:
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold

X_reshape_2 = X.reshape(X.shape[0], X.shape[1] * X.shape[2])

X_SS_2 = StandardScaler().fit_transform(X_reshape_2)
y_2 = LabelEncoder().fit_transform(labels)

imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
X_SS_2 = imputer.fit_transform(X_SS_2)

skf = StratifiedKFold(n_splits=3)
rfe = RFECV(SVC(kernel="linear"), step=0.01, min_features_to_select=100, cv=skf, scoring='accuracy', n_jobs=8)
X_rfe = rfe.fit_transform(X_SS_2, y_2)
X_rfe.shape


(160, 100)

In [230]:
X_train_full, X_test_full, y_train_full, y_test_full = train_test_split(X_rfe, y, test_size=0.2, random_state=42)

svm_full_model = SVC(kernel='linear', C=5, probability=True, gamma='auto').fit(X_train_full, y_train_full)
y_full_pred = svm_full_model.predict(X_test_full)

print("Accuracy: %.2f %%" % (accuracy_score(y_test_full, y_full_pred) * 100))
# 21.88

Accuracy: 53.12 %


In [234]:
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import MinMaxScaler

imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
X_KBest = imputer.fit_transform(X_reshape_2)

# Normalizando o X
X_KBest_norm = MinMaxScaler().fit_transform(X_KBest)

X_full_KBest = SelectKBest(f_classif, k=100).fit_transform(X_KBest_norm, y)
X_full_KBest.shape

(160, 100)

In [239]:
X_train_full, X_test_full, y_train_full, y_test_full = train_test_split(X_full_KBest, y, test_size=0.3, random_state=42)

svm_full_model = SVC(kernel='linear').fit(X_train_full, y_train_full)
y_full_pred = svm_full_model.predict(X_test_full)

print("Accuracy: %.2f %%" % (accuracy_score(y_test_full, y_full_pred) * 100))

Accuracy: 31.25 %


In [245]:
from sklearn.ensemble import RandomForestClassifier

X_train_rfc, X_test_rfc, y_train_rfc, y_test_rfc = train_test_split(X_full_KBest, y, test_size=0.3, random_state=42)

rfc = RandomForestClassifier(n_estimators=1000, random_state=42, n_jobs=8)
rfc.fit(X_train_rfc, y_train_rfc)

# assuming X_test is your test feature matrix
y_pred = rfc.predict(X_test_rfc)

# calculate accuracy
print("Accuracy: %.2f %%" % (accuracy_score(y_test_rfc, y_pred) * 100))


Accuracy: 25.00 %
